# Draw

The task of the robot is to drive around without collision to draw on a canvas.

The trained network should estimate in which direction it is save to drive:

- forward : both tracks have same speed value
- right : left track has set speed, right track is set 0
- left : right track has set speed, left track is set 0
- turn_right : left track has set speed, right has set -speed
- turn_left : right track has set speed, left has set -speed

To get image data to train the resnet18 the following script will collect images when the robot is driving (controlled by the gamepad).

The joystick values will be calculated into the above values.

In [ ]:
import ipywidgets.widgets as widgets
import traitlets
from jetbot import Robot, Camera, bgr8_to_jpeg
import os
import uuid
import time
from threading import Thread, Event

controller = widgets.Controller(index=0)  # replace with index of your controller
display(controller)

In [ ]:
robot = Robot()

speed = 0.5

# Create output directories
output_dirs = {
    'forward': 'dataset/forward',
    'right': 'dataset/right',
    'left': 'dataset/left',
    'turn_right': 'dataset/turn_right',
    'turn_left': 'dataset/turn_left'
}

for directory in output_dirs.values():
    os.makedirs(directory, exist_ok=True)

def classify_movement(left_value, right_value):
    if left_value > 0.5 and right_value > 0.5:
        return 'forward'
    elif left_value < 0.5 and right_value > 0.5:
        return 'left'
    elif left_value > 0.5 and right_value < 0.5:
        return 'right'
    elif left_value > -0.5 and right_value > 0.5:
        return 'turn_left'
    elif left_value > 0.5 and right_value > -0.5:
        return 'turn_right'
    else:
        return None

def control_robot(change):
    left_value = controller.axes[1].value
    right_value = controller.axes[3].value

    movement = classify_movement(left_value, right_value)
    if movement == 'forward':
        robot.forward(speed)
    elif movement == 'right':
        robot.right(speed)
    elif movement == 'left':
        robot.left(speed)
    elif movement == 'turn_right':
        robot.set_motors(speed, -speed)
    elif movement == 'turn_left':
        robot.set_motors(-speed, speed)
    else:
        robot.stop()

controller.axes[1].observe(control_robot, names='value')
controller.axes[3].observe(control_robot, names='value')

camera = Camera.instance(width=224, height=224)
image = widgets.Image(format='jpeg', width=224, height=224)
camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

display(image)

is_capturing = False
capture_event = Event()

# Create widgets for displaying image count and recording status
image_counts = {key: widgets.IntText(layout=button_layout, value=len(os.listdir(path))) for key, path in output_dirs.items()}
recording_status = widgets.Label(value='Recording: OFF')

def update_image_counts():
    for key, path in output_dirs.items():
        image_counts[key].value = len(os.listdir(path))

def update_recording_status():
    recording_status.value = f"Recording: {'ON' if is_capturing else 'OFF'}"

def capture_loop():
    global is_capturing, capture_event
    while True:
        capture_event.wait()
        left_value = controller.axes[1].value
        right_value = controller.axes[3].value
        movement = classify_movement(left_value, right_value)

        if movement:
            directory = output_dirs[movement]
            file_path = os.path.join(directory, str(uuid.uuid1()) + '.jpg')
            with open(file_path, 'wb') as f:
                f.write(image.value)
            update_image_counts()

        time.sleep(0.1)

capture_thread = Thread(target=capture_loop)
capture_thread.start()

def toggle_capture(change):
    global is_capturing, capture_event
    if change['new']:
        is_capturing = not is_capturing
        update_recording_status()
        if is_capturing:
            capture_event.set()
        else:
            capture_event.clear()

controller.buttons[5].observe(toggle_capture, names='value')

display(image)

display(recording_status)

button_layout = widgets.Layout(width='128px', height='64px')

# Create widgets for displaying image count and recording status
image_counts = {key: widgets.IntText(layout=button_layout, value=len(os.listdir(path))) for key, path in output_dirs.items()}

close the camera conneciton properly so that we can use the camera in the later notebook.

In [ ]:
camera.stop()